In [ ]:
"""
Purpose: To Run the neuron preprocessing


"""

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

# configuring the virtual module

In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
#time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2020-11-22 21:18:00,439 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 21:18:00,440 - settings - Setting database.user to celiib
INFO - 2020-11-22 21:18:00,441 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 21:18:00,445 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-22 21:18:00,446 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 21:18:00,456 - connection - Connect

Sleeping 9 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


# Defining Our Table

In [3]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [4]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 4602


In [5]:
#so that it will have the adapter defined
from datajoint_utils import *

In [ ]:
#minnie.Decomposition.drop()
#schema.external['decomposition'].delete(delete_external_files=True)

In [ ]:
import numpy as np
import time
decimation_version = 0
decimation_ratio = 0.25

@schema
class Decomposition(dj.Computed):
    definition="""
    -> minnie.Decimation.proj(decimation_version='version')
    ---
    decomposition: <decomposition>
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branche

    
    """

    key_source =  ((minnie.Decimation).proj(decimation_version='version') & 
                            "decimation_version=" + str(decimation_version) &
                       f"decimation_ratio={decimation_ratio}" &  (minnie.BaylorSegmentCentroid() & "multiplicity>0").proj())
    

    def make(self,key):
        """
        Pseudocode for process:

        1) Get the segment id from the key
        2) Get the decimated mesh
        3) Get the somas info
        4) Run the preprocessing
        5) Calculate all starter stats
        6) Save the file in a certain location
        7) Pass stats and file location to insert
        """
        #1) Get the segment id from the key
        segment_id = key["segment_id"]
        description = str(key['decimation_version']) + "_25"
        print(f"\n\n----- Working on {segment_id}-------")
        global_start = time.time()
        
        #2) Get the decimated mesh
        current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

        #3) Get the somas info *************************** Need to change this when actually run *******************
        somas = du.get_soma_mesh_list(segment_id) 
        print(f"somas = {somas}")
        #4) Run the preprocessing


        total_neuron_process_time = time.time()

        print(f"\n--- Beginning preprocessing of {segment_id}---")
        recovered_neuron = neuron.Neuron(
        mesh = current_neuron_mesh,
        somas = somas,
        segment_id=segment_id,
        description=description,
        suppress_preprocessing_print=False,
        suppress_output=False,
        calculate_spines=True,
        widths_to_calculate=["no_spine_median_mesh_center"]

                )

        print(f"\n\n\n---- Total preprocessing time = {time.time() - total_neuron_process_time}")


        #5) Don't have to do any of the processing anymore because will do in the neuron object
        stats_dict = recovered_neuron.neuron_stats()



        #6) Save the file in a certain location
        save_time = time.time()
        ret_file_path = recovered_neuron.save_compressed_neuron(output_folder=str(du.get_decomposition_path()),
                                          return_file_path=True,
                                         export_mesh=False,
                                         suppress_output=True)

        ret_file_path_str = str(ret_file_path.absolute()) + ".pbz2"
        print(f"Save time = {time.time() - save_time}")



        #7) Pass stats and file location to insert
        new_key = dict(key,
                       decomposition=ret_file_path_str,
                       n_vertices=len(current_neuron_mesh.vertices),
                       n_faces=len(current_neuron_mesh.faces)
                      )
        new_key.update(stats_dict)

        self.insert1(new_key, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {segment_id} = {time.time() - global_start} ------")
    

# Cleaning the external files

In [ ]:
(minnie.schema.jobs & "table_name='__decomposition'").delete()
#((schema.jobs & "table_name = '__decomposition'") & "timestamp>'2020-11-16 00:26:00'").delete()

In [ ]:
import preprocessing_vp2 as pre
import compartment_utils as cu
cu = reload(cu)
pre = reload(pre)

In [ ]:
import time
import random

start_time = time.time()
#time.sleep(random.randint(0, 900))
print('Populate Started')
Decomposition.populate(reserve_jobs=True, suppress_errors=False, order='random')
print('Populate Done')

print(f"Total time for Decomposition populate = {time.time() - start_time}")

# Clearing Files

In [ ]:
curr_path = Path("/mnt/dj-stor01/platinum/minnie65/02/decomposition/")
for f in curr_path.iterdir():
    print(f'{f}')
    #f.unlink()
    

# Checking the Table

In [ ]:
minnie.Decomposition()

In [ ]:
du = reload(du)
from datajoint_utils import *
my_dj_neuron = (minnie.Decomposition & "segment_id=864691134221889045").fetch1("decomposition")

import neuron_visualizations as nviz
nviz.visualize_neuron(my_dj_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")

# Debugging the neuron creation

In [11]:
#1) Get the segment id from the key
segment_id = 864691136723392509
description = "0_25"
print(f"\n\n----- Working on {segment_id}-------")
global_start = time.time()

#2) Get the decimated mesh
current_neuron_mesh = du.fetch_segment_id_mesh(segment_id)

#3) Get the somas info *************************** Need to change this when actually run *******************
somas = du.get_soma_mesh_list(segment_id) 
print(f"somas = {somas}")
#4) Run the preprocessing


total_neuron_process_time = time.time()



----- Working on 864691136723392509-------
somas = [[<trimesh.Trimesh(vertices.shape=(23452, 3), faces.shape=(44881, 3))>, <trimesh.Trimesh(vertices.shape=(7256, 3), faces.shape=(14137, 3))>], array([369.0962, 369.0962]), array([0.691, 0.771])]


In [12]:
import preprocessing_vp2 as pre
import soma_extraction_utils as sm

import neuron
tu = reload(tu)
du = reload(du)
sm = reload(sm)
pre = reload(pre)
import compartment_utils as cu
cu = reload(cu)

neuron = reload(neuron)
print(f"\n--- Beginning preprocessing of {segment_id}---")
recovered_neuron = neuron.Neuron(
mesh = current_neuron_mesh,
somas = somas,
segment_id=segment_id,
description=description,
suppress_preprocessing_print=False,
suppress_output=False,
calculate_spines=True,
widths_to_calculate=["no_spine_median_mesh_center"]

        )


--- Beginning preprocessing of 864691136723392509---
--- 0) Having to preprocess the Neuron becuase no preprocessed data
Please wait this could take a while.....
Skipping the hole filling
use_meshafterparty = True
Using pre-computed somas: soma_mesh_list = [<trimesh.Trimesh(vertices.shape=(23452, 3), faces.shape=(44881, 3))>, <trimesh.Trimesh(vertices.shape=(7256, 3), faces.shape=(14137, 3))>]
Soma List = [<trimesh.Trimesh(vertices.shape=(23452, 3), faces.shape=(44881, 3))>, <trimesh.Trimesh(vertices.shape=(7256, 3), faces.shape=(14137, 3))>]
soma_mesh_list_centers = [array([1336046.9787225 ,  948815.04275969,  673971.43516971]), array([1285732.74972437,  961093.7603914 ,  665071.4768054 ])]
xvfb-run -n 4602 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging/temp/neuron_44004.off -o /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging/temp/neuron_44004_fill_holes.off -s /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:17<00:00, 17.91s/it]

Time for 1st pass MP skeletonization: 20.517661333084106
branches_touching_root = [9]
length of Graph = 34456
Working on path [11016. 11025. 11034. 11044. 11053. 11065. 11078. 11081.]
path_degrees = [3, 2, 2, 2, 2, 2, 2, 3]
Working on path [17557. 17580. 17602. 17615. 17634. 17640.]
path_degrees = [3, 2, 2, 2, 2, 3]
Working on path [18877. 18879. 18881.]
path_degrees = [3, 2, 3]
Working on path [23227. 23243. 23257. 23282. 23297.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [29283. 29277. 29278. 29284. 29298. 29313. 29320.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [34460. 29303. 29334. 29343.]
path_degrees = [4, 2, 2, 3]
Working on path [30119. 30130. 30143. 30153. 30156.]
path_degrees = [3, 2, 2, 2, 3]
Working on path [30316. 30370. 30407. 30427. 30452. 30478. 30493.]
path_degrees = [3, 2, 2, 2, 2, 2, 3]
Working on path [34463. 30548. 30587. 30621. 30659. 30702. 30711.]
path_degrees = [4, 2, 2, 2, 2, 2, 3]
max(kept_branches_idx) = 192, len(kept_branches_idx) = 184
empty


AFTER face_lookup_resolved_test


...


AFTER face_lookup_resolved_test



Decomposing first pass: 1.403672456741333
Attempting to use MeshAfterParty Skeletonization and Mesh Correspondence
Another print
Divinding into MP and MAP pieces: 6.67572021484375e-05
Total time for MAP sublimb processing 2.6226043701171875e-06
---- Working on MP Decomposition #0 ----
Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece 
MP filtering soma verts: 6.151199340820312e-05
Fixing Possible Soma Extension Branch for Sublimb 0
Total time for mesh KDTree = 0.004584074020385742
sbv[0].reshape(-1,3) = [[1289265.   960885.2  669935.6]]
closest_sk_pt_coord BEFORE = [1289327.47697725  960649.73567972  671086.09169848]
current_skeleton.shape = (52, 2, 3)
Current stitch point was not a branch or endpoint, shortest_path_length to one = 2299.9783802438924
change_status for create soma extending pieces = False
closest_sk_pt_coord AFTER = [1289327.47697725  960649.73567972  671086.09169848]
Adding new branch to skeleton
border_average_coordinate = [




--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (2, 3)
empty_indices % = 0.19054054054054054
 conflict_indices % = 0.16756756756756758



AFTER face_lookup_resolved_test
Took 59 iterations to expand the label back
empty_indices % = 0.0
 conflict_indices % = 0.0
Doing No Color conflicts iteration 2 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 3 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 4 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 5 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 6 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 7 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 8 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 9 because missing_labels = {0, 1} 
Doing No Color conflicts iteration 10 because missing_labels = {0, 1} 
leftover_labels = [2]
no_missing_labels = [0 1 2]
missing_labels = {0, 1}
Saved object at /notebooks/Platinum_Datajoint/Neuron_Processing/Debugging/curr_limb_mesh.pbz2
File size is 0.063252 MB
Saved object at /notebooks/Platinum_Datajoint/Neuro

Exception: missing labels was not resolved

In [8]:
import neuron_visualizations as nviz

In [9]:
nviz.visualize_neuron(recovered_neuron,
                     visualize_type=["mesh","skeleton"],
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton
Working on  new stand alone scatter points


In [10]:
du.plot_decimated_mesh_with_somas(864691136723392509,minnie=minnie)